In [1]:
import os
import numpy as np
import random
import datetime
import subprocess
import shutil

In [2]:
#this function will return the length in seconds of a file when given the full file path
def get_length(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return round(float(result.stdout))

#this function will return a list of every file in a directory when given the path to a particular folder
def get_full_path(path):
    arr=[]
    for file in os.listdir(path):
        if not file.startswith('.'):
            location=path+file
            if os.path.isdir(location):
                continue
            else:
                arr.append(location)
    return np.array(arr)

#this function will return the lengths for a series of file when given a list of file paths
def length_vids(files):
    return np.array([get_length(entry) for entry in files])

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

In [3]:
months=np.array(['January','February','March','April','May','June','July','August','September',
       'October','November','December'])

mydate=datetime.datetime.now()
curmonth=mydate.strftime("%B")

In [4]:
#pull in and get the full paths of all the files in various folders
commercial_full_path=get_full_path('E:/Archived Bumps & Commercials/Commercials/CN/')
cn_promo_full_path=get_full_path('E:/Archived Bumps & Commercials/Cartoon Network/Promo_Commercials/')
ccf_promo_full_path=get_full_path('E:/Archived Bumps & Commercials/Cartoon Network/Promo_Commercials/CCF/Powerhouse Fridays/')
toonami_promo_full_path=get_full_path('E:/Archived Bumps & Commercials/Toonami/Promo_Commercials/TOM 2/')
boomerang_bumps=get_full_path('E:/Archived Bumps & Commercials/Boomerang/Bumps/')
station_ids=get_full_path('E:/Archived Bumps & Commercials/Cartoon Network/Bumps/Powerhouse/2 - Station IDs/')

In [5]:
#read in past broadcasts
#past_broadcasts=get_full_path('D:/Block_Dump/')
#past_broadcasts=past_broadcasts[np.array(['Boomerang' in block and 'txt' in block for block in past_broadcasts])]

#open up all past broadcasts and create a list of show episodes that were used
#for i,cast in enumerate(past_broadcasts):
#    with open(cast) as f:
#        lines=np.array(f.readlines())
#        shows_used=lines[np.array(['Broadcast_Shows' in line for line in lines])]
        
#    if i==0:
#        past_shows=shows_used
#    else:
#        past_shows=np.concatenate((past_shows,shows_used))
        
#show_uqid=np.array([])

#for entry in past_shows:
#    mp4_loc=entry.find('/mp4')
#    show_loc=find(entry[:mp4_loc],'/')[-1]+1
    
#    show_uqid=np.append(show_uqid,entry[show_loc:mp4_loc])

#master_path='D:/Block_Dump/past_eps_used/'

#for i,uqid in enumerate(show_uqid):
#    with open(master_path+uqid+'.txt','r+') as f:
#        f.seek(0)
#        contents=f.readlines()
        
#    with open(master_path+uqid+'.txt','w+') as f:
#        if past_shows[i][6:] not in contents:
#            contents.insert(-1,past_shows[i][6:])
#        contents = "".join(contents)
#        f.write(contents)

In [6]:
#remove commercials, promos that occur in different parts of the year

curmonth_toonami_promo_full_path=np.array([promo for promo in toonami_promo_full_path if curmonth in promo])
curmonth_commercial_full_path=np.array([promo for promo in commercial_full_path if curmonth in promo])
curmonth_cn_promo_full_path=np.array([promo for promo in cn_promo_full_path if curmonth in promo])
curmonth_ccf_promo_full_path=np.array([promo for promo in ccf_promo_full_path if curmonth in promo])

for month in months:
    toonami_promo_full_path=np.array([promo for promo in toonami_promo_full_path if month not in promo])
    commercial_full_path=np.array([promo for promo in commercial_full_path if month not in promo])
    cn_promo_full_path=np.array([promo for promo in cn_promo_full_path if month not in promo])
    ccf_promo_full_path=np.array([promo for promo in ccf_promo_full_path if month not in promo])
    
toonami_promo_full_path=np.append(toonami_promo_full_path,curmonth_toonami_promo_full_path)
commercial_full_path=np.append(commercial_full_path,curmonth_commercial_full_path)
cn_promo_full_path=np.append(cn_promo_full_path,curmonth_cn_promo_full_path)
ccf_promo_full_path=np.append(ccf_promo_full_path,curmonth_ccf_promo_full_path)

In [7]:
#get the lengths for all video files in each subdirectory from above
commercial_lengths=length_vids(commercial_full_path)
cn_promo_lengths=length_vids(cn_promo_full_path)
ccf_promo_lengths=length_vids(ccf_promo_full_path)
toonami_promo_lengths=length_vids(toonami_promo_full_path)

In [8]:
#randomly select a set amount of promos that will be repeated and scattered throughout the broadcast
toonami_selected_promos=random.sample(list(toonami_promo_full_path),5)

for keyword in ['Schedule','Sweepstake','General_Promo','Rising Sun','Lineup']:
    if len(np.argwhere(np.array([keyword in single_item for single_item in toonami_selected_promos])==True))>1:
        toonami_selected_promos.remove(toonami_selected_promos[np.argwhere(np.array([keyword in single_item for single_item in toonami_selected_promos])==True)[0,0]])
        new_sample=random.sample(list(toonami_promo_full_path),1)
        if keyword not in new_sample[0]:
            toonami_selected_promos.append(new_sample[0])

cn_selected_promos=random.sample(list(cn_promo_full_path),10)
ccf_selected_promo=random.sample(list(ccf_promo_full_path),1)[0]

toonami_selected_promos,cn_selected_promos,ccf_selected_promo

(['E:/Archived Bumps & Commercials/Toonami/Promo_Commercials/TOM 2/Toonami_Lineup_Promo.mp4',
  'E:/Archived Bumps & Commercials/Toonami/Promo_Commercials/TOM 2/Toonami - Rising Sun Promo (Tom 2) [1080p HD]-YxtJZj0hxrE.mp4',
  'E:/Archived Bumps & Commercials/Toonami/Promo_Commercials/TOM 2/Toonami_Midnight_Run_Schedule_Promo_DB_DBZ.mp4',
  'E:/Archived Bumps & Commercials/Toonami/Promo_Commercials/TOM 2/Toonami_SlyCooper_Promo.mp4',
  'E:/Archived Bumps & Commercials/Toonami/Promo_Commercials/TOM 2/Toonami_MidnightRun_Promo_1.mp4'],
 ['E:/Archived Bumps & Commercials/Cartoon Network/Promo_Commercials/Wonder Woman and Johnny Bravo 1998 1999 2000 2001 2002.mp4',
  'E:/Archived Bumps & Commercials/Cartoon Network/Promo_Commercials/KND next episode 2003.mp4',
  'E:/Archived Bumps & Commercials/Cartoon Network/Promo_Commercials/Powerpuff Girls Reviews 1999.mp4',
  'E:/Archived Bumps & Commercials/Cartoon Network/Promo_Commercials/Dexter - Name spelling song 2000 2001.mp4',
  'E:/Archived B

In [9]:
master_order=[]

In [10]:
#this function is responsible for generating commercial blocks
def commercial_generator(length,probs,master_order,toonami_selected_promos=toonami_selected_promos,
                        toonami_promo_lengths=toonami_promo_lengths,toonami_promo_full_path=toonami_promo_full_path,
                        cn_selected_promos=cn_selected_promos,cn_promo_lengths=cn_promo_lengths,
                        cn_promo_full_path=cn_promo_full_path,commercial_full_path=commercial_full_path,
                        commercial_lengths=commercial_lengths,ccf_selected_promo=ccf_selected_promo,
                        ccf_promo_length=ccf_promo_lengths,ccf_promo_full_path=ccf_promo_full_path
                        ):

    commercials_list=[]

    commercial_block_size=length

    #assign probality dictionary to a temp variable
    flag_dict=probs

    #while there's still at least 10 seconds left in the commercial block, continue to loop through and add more commercials
    while commercial_block_size>10:

        #calculate the remaining probability after subtracting the other probabilities
        remainder=1-(flag_dict['toonami']+flag_dict['cn']+flag_dict['ccf'])
        #assign all probabilities 
        probabilities=[flag_dict['toonami'],flag_dict['cn'],flag_dict['ccf'],remainder]
        #randomly select which 'type' of commercial will be next from these 4 specific options - this can be modified accordingly
        commercial_type=random.choices(['toonami','cn','ccf','commercial'],weights=probabilities)[0]

        #if the selected commercial type is anything other than a normal commercial, reset the probability of selecting
        #that commercial type again to zero to avoid repeats
        if commercial_type!='commercial':
            flag_dict[commercial_type]=0

        #check the commercial type and cross reference the selected promo against the lengths calculated earlier
        if commercial_type=='toonami':
            commercial=random.choice(toonami_selected_promos)
            diff=int(toonami_promo_lengths[toonami_promo_full_path==commercial])
        elif commercial_type=='cn':
            commercial=random.choice(cn_selected_promos)
            diff=int(cn_promo_lengths[cn_promo_full_path==commercial])
        elif commercial_type=='ccf':
            commercial=ccf_selected_promo
            diff=int(ccf_promo_length[ccf_promo_full_path==commercial])
        elif commercial_type=='commercial':
            commercial=random.choice(commercial_full_path)
            diff=int(commercial_lengths[commercial_full_path==commercial])

        #subtract the remaining time left in the commercial block by the length of the commercial/promo just selected
        commercial_block_size=commercial_block_size-diff
        
        #if the addition of the new promo causes the commercial block to run 10 seconds over the specified time, then skip 
        #it and select another one
        if commercial_block_size<-10:
            commercial_block_size=commercial_block_size+diff
        #if the commercial has already been selected for the broadcast then skip it and select another one
        elif commercial in commercials_list:
            commercial_block_size=commercial_block_size+diff
        #if the commercial is part of the last 30 files in the order, toss it out for a new one to keep it fresh
        elif commercial in master_order[-30:]:
            commercial_block_size=commercial_block_size+diff
        #otherwise if the earlier 3 criteria aren't met, add the commercial to the list
        else:
            commercials_list.append(commercial)
            
            #once a commercial is added to the list, adjust the probabilities so they tick upwards
            #ccf probabilities tick up at 1/4 the rate of the other promos just because - this can be adjusted based 
            #on personal preference
            for ctype in ['toonami','cn','ccf']:
                if ctype=='ccf':
                    flag_dict[ctype]+=0.001
                elif ctype=='toonami':
                    flag_dict[ctype]+=0.005
                else:
                    flag_dict[ctype]+=0.015
                
        #print(commercial_block_size,diff)
        #print(probabilities)

    #once done, this function will return a list of commercials, and the probabilities so that they can be carried
    #over as input for the next commercial break
    return(commercials_list,flag_dict)

In [11]:
#establish starting probabilities. settled on an even 5% per category. can be modified by any value between 0 and 1
flag_dict={'toonami':0.05,'cn':0.05,'ccf':0.05}

In [12]:
multi_show_dict={
    'Atom Ant':['Atom Ant','Precious Pupp','Hillbilly Bears'],
    'Quickdraw Mcgraw':['Quickdraw Mcgraw','Auggie Doggie','Snooper and Blabber'],
    'Magilla Gorilla':['Magilla Gorilla','Ricochet Rabbit','Punkin Puss'],
    'Huckleberry Hound':['Huckleberry Hound','Pixie Dixie and Mr Jinks','Hokey Wolf'],
    'Peter Potamus':['Peter Potamus','Breezly and Sneezly','Yippee Yappee Yahooey'],
    'Secret Squirrel':['Secret Squirrel','Squiddly Diddly','Winsome Witch'],
    'Yogi Bear':['Yogi Bear','Snagglepuss','Yakky Doodle']
}

In [13]:
for i,id_cat in enumerate(['Flintstones','Huckleberry Hound','Jetsons','Captain Planet','Hong Kong Phooey','Quick Draw','Secret Squirrel'\
              ,'Smurfs','Top Cat','Yogi Bear']):
    if i==0:
        filtered_ids=station_ids[np.array([id_cat in ids for ids in station_ids])]
    else:
        filtered_ids=np.concatenate((filtered_ids,station_ids[np.array([id_cat in ids for ids in station_ids])]))

In [14]:
#list of shows you want in block in order
shows=['New Scooby Doo Movies','Jetsons','Atom Ant']

master_order=[]

master_order.append('E:/Archived Bumps & Commercials/Boomerang/Bumps/Up Next - Boomerang.mp4')

#generate a commercial break using the function above, specificying the length of the block in seconds followed by
#the probabilities and then append each entry in the commercial block to the master file
tmp_cblock,flag_dict=commercial_generator(75,flag_dict,master_order)
for tmp_c in tmp_cblock:
    master_order.append(tmp_c)

master_order.append('E:/Archived Bumps & Commercials/Boomerang/Bumps/Boomerang General Ident Cartoon Network Website ALl Coming back to you.mp4')
if os.path.isfile('E:/Archived Bumps & Commercials/Boomerang/Bumps/Up Next - '+shows[0]+'.mp4')==True:
    master_order.append('E:/Archived Bumps & Commercials/Boomerang/Bumps/Up Next - '+shows[0]+'.mp4')

for i,show in enumerate(shows):
    
    master_order.append(random.choice(filtered_ids))
    
    #select a lead in intro for the show
    #if a unique intro does not exist, insert a generic one
    show_intro=boomerang_bumps[np.array([show+' Intro' in bump for bump in boomerang_bumps])]
    if len(show_intro)<1:
        neutral_intros=boomerang_bumps[np.array(['Neutral Any' in bump for bump in boomerang_bumps])]
        master_order.append(random.choice(neutral_intros))
    else:
        master_order.append('E:/Archived Bumps & Commercials/Boomerang/Bumps/'+show+' Intro.mp4')
    
    #check to see if the show was syndicated with other shows
    #if it is, select random episodes from each one in the proper order to be run together
    if show not in multi_show_dict:        
        show_files=get_full_path('E:/Broadcast_Shows/Boomerang/'+show+'/mp4/Subsections/')
        
        #look through the array of shows in previous broadcasts and isolate episodes for current show and remove 
        #episodes that have already been used
        master_path='D:/Block_Dump/past_eps_used/'
        with open(master_path+show+'.txt','r+') as f:
            f.seek(0)
            past_episodes_used=f.readlines()
        if len(past_episodes_used)>0:
            for past_episode in past_episodes_used:
                index=np.argwhere(show_files==past_episode[:-2])[0,0]
                show_files=np.delete(show_files,index)  
        
        ep_selector=random.choice(show_files)
        show_segments=show_files[np.array([ep_selector[ep_selector.find('Subsection')+12:-6] in segment for segment in show_files])==True]
    else:
        for placemarker,feature in enumerate(multi_show_dict[show]):
            show_files=get_full_path('E:/Broadcast_Shows/Boomerang/'+feature+'/mp4/Subsections/')
            
            #look through the array of shows in previous broadcasts and isolate episodes for current show and remove 
            #episodes that have already been used
            master_path='D:/Block_Dump/past_eps_used/'
            with open(master_path+feature+'.txt','r+') as f:
                f.seek(0)
                past_episodes_used=f.readlines()
            if len(past_episodes_used)>0:
                for past_episode in past_episodes_used:
                    index=np.argwhere(show_files==past_episode[:-2])[0,0]
                    show_files=np.delete(show_files,index)  

            ep_selector=random.choice(show_files)
            if placemarker==0:
                show_segments=show_files[np.array([ep_selector[ep_selector.find('Subsection')+12:-6] in segment for segment in show_files])==True]
            else:
                show_segments=np.concatenate((show_segments,show_files[np.array([ep_selector[ep_selector.find('Subsection')+12:-6] in segment for segment in show_files])==True]))
                
    for j,segment in enumerate(show_segments):
        #add the first part of the show to the block order
        master_order.append(segment)
        
        if show in multi_show_dict and j==0:
            interstitial=boomerang_bumps[np.array([show+' Interstitial' in bump for bump in boomerang_bumps])]
            
            if len(interstitial)<1:
                interstitial=boomerang_bumps[np.array(['Neutral Any' in bump for bump in boomerang_bumps])]
                
            master_order.append(random.choice(interstitial))
            
            continue
            
        transition_bumps_in=boomerang_bumps[np.array([show+' Fade In' in bump for bump in boomerang_bumps])]
        transition_bumps_out=boomerang_bumps[np.array([show+' Fade Out' in bump for bump in boomerang_bumps])]
        
        if len(transition_bumps_in)<1:
            transition_bumps_in=boomerang_bumps[np.array(['Neutral Fade In' in bump for bump in boomerang_bumps])]
        if len(transition_bumps_out)<1:
            transition_bumps_out=boomerang_bumps[np.array(['Neutral Fade Out' in bump for bump in boomerang_bumps])]

        if segment!=show_segments[-1]:
            master_order.append(random.choice(transition_bumps_out))
            
            tmp_cblock,flag_dict=commercial_generator(210,flag_dict,master_order)
            for tmp_c in tmp_cblock:
                master_order.append(tmp_c)

            master_order.append(random.choice(transition_bumps_in))
            
        elif (segment==show_segments[-1]):
            
            ending_bump=boomerang_bumps[np.array([show+' End' in bump for bump in boomerang_bumps])]
            if len(ending_bump)>0:
                master_order.append(random.choice(ending_bump))
            
            if i<(len(shows)-1):
                if os.path.isfile('E:/Archived Bumps & Commercials/Boomerang/Bumps/Up Next - '+shows[i+1]+'.mp4')==True:
                    master_order.append('E:/Archived Bumps & Commercials/Boomerang/Bumps/Up Next - '+shows[i+1]+'.mp4')
                
                tmp_cblock,flag_dict=commercial_generator(210,flag_dict,master_order)
                for tmp_c in tmp_cblock:
                    master_order.append(tmp_c)
            else:
                master_order.append('E:/Archived Bumps & Commercials/Boomerang/Bumps/Neutral End.mp4')

                #master_order.append('E:/Archived Bumps & Commercials/Boomerang/Bumps/Boomerang General Ident Cartoon Network Website ALl Coming back to you.mp4')

In [15]:
master_order

['E:/Archived Bumps & Commercials/Boomerang/Bumps/Up Next - Boomerang.mp4',
 'E:/Archived Bumps & Commercials/Commercials/CN/2004 Royal Caribbean Cruise Europe 2004.mp4',
 'E:/Archived Bumps & Commercials/Commercials/CN/2000_Kraft Macaroni and Cheese - I Got the Blues 2000.mp4',
 'E:/Archived Bumps & Commercials/Commercials/CN/2001_Popsicle Scribblers 2001.mp4',
 'E:/Archived Bumps & Commercials/Boomerang/Bumps/Boomerang General Ident Cartoon Network Website ALl Coming back to you.mp4',
 'E:/Archived Bumps & Commercials/Cartoon Network/Bumps/Powerhouse/2 - Station IDs/Jetsons - 3D house 2003.mp4',
 'E:/Archived Bumps & Commercials/Boomerang/Bumps/Neutral Any 1.mp4',
 'E:/Broadcast_Shows/Boomerang/New Scooby Doo Movies/mp4/Subsections/Ghostly, Ghostly Town (Scooby meets The Three Stooges)_000.mp4',
 'E:/Archived Bumps & Commercials/Boomerang/Bumps/Neutral Fade Out 2.mp4',
 'E:/Archived Bumps & Commercials/Toonami/Promo_Commercials/TOM 2/Toonami_MidnightRun_Promo_1.mp4',
 'E:/Archived Bu

In [16]:
save_shows=False

if save_shows==True:

    past_shows=np.array(master_order)[np.array(['Broadcast_Shows' in line for line in master_order])]

    show_uqid=np.array([])

    for entry in past_shows:
        mp4_loc=entry.find('/mp4')
        show_loc=find(entry[:mp4_loc],'/')[-1]+1

        show_uqid=np.append(show_uqid,entry[show_loc:mp4_loc])

    master_path='D:/Block_Dump/past_eps_used/'

    for i,uqid in enumerate(show_uqid):
        with open(master_path+uqid+'.txt','r+') as f:
            f.seek(0)
            contents=f.readlines()

        with open(master_path+uqid+'.txt','w+') as f:
            if past_shows[i][6:] not in contents:
                contents.insert(-1,past_shows[i]+"'\n")
            contents = "".join(contents)
            f.write(contents)

In [17]:
np.savetxt('D:/Block_Dump/Boomerang_008.txt',np.array(['file '+"'"+entry+"'" for entry in master_order]),fmt='%s')